In [ ]:
import os
import cv2
import pickle

from imutils import paths
import face_recognition

In [ ]:
testing_dataset = "dataset/testing"

In [ ]:
# find path of xml file containing haarcascade file
cascPathface = os.path.dirname(
 cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
# load the harcaascade in the cascade classifier
faceCascade = cv2.CascadeClassifier(cascPathface)
# load the known faces and embeddings saved earlier
data = pickle.loads(open('face_enc_live', "rb").read())

In [ ]:
def identify_face(image):
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #convert image to Greyscale for haarcascade
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    
    encodings = face_recognition.face_encodings(rgb)

    if len(encodings) > 1:
        print("Found more than one encoding. Most likely, the image has multiple faces")
        return None, None
    elif len(encodings) == 0:
        print("No encoding match found for the given face.")
        return None, None
    
    names = []
    matches = face_recognition.compare_faces(data["encodings"], encodings[0])

    if True in matches:
        #Find positions at which we get True and store them
        matchedIdxs = [i for (i, b) in enumerate(matches) if b]
        counts = {}
        # loop over the matched indexes and maintain a count for
        # each recognized face face
        for i in matchedIdxs:
            #Check the names at respective indexes we stored in matchedIdxs
            name = data["names"][i]
            #increase count for the name we got
            counts[name] = counts.get(name, 0) + 1
            #set name which has highest count
            name = max(counts, key=counts.get)


        # update the list of names
        names.append(name)
        if len(names) > 1:
            print("Found more than one match for facial encoding")
            return None, None
        
        # loop over the recognized faces
        for ((x, y, w, h), name) in zip(faces, names):
            # rescale the face coordinates
            # draw the predicted face name on the image
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, name, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
             0.75, (0, 255, 0), 2)
    else:
        print("No encoding matches found. Possibly a new face")
        return None, None
#     cv2.imshow("Frame", image)
#     cv2.waitKey(0)
    if len(names) == 0:
        return None
    return names[0], image

Labels encoded

In [ ]:
names = sorted(list(set(data['names'])))
for name in names:
    print(name, '\n')

In [ ]:
prediction_status = {'correct': 0, 'incorrect': 0, 'unidentified': 0}

In [ ]:
# testing on validation dataset

imagePaths = list(paths.list_images(testing_dataset))

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    try:
        name = imagePath.split(os.path.sep)[-2]
    except ValueError:
        name = "Unknown"
    image = cv2.imread(imagePath)
    pred_name, pred_image = identify_face(image)
    if pred_name is None:
        print(f"Face could not be recognized for {imagePath.split(os.path.sep)[-1]}")
    else:
        print(f"Original face: {name}; Identified face: {pred_name}")
        if pred_name != name:
            print("Wrong face match found.")
    if pred_name == name:
        prediction_status['correct'] += 1
    elif pred_name == None:
        prediction_status['unidentified'] += 1
    elif pred_name != name:
        prediction_status['incorrect'] += 1
    print()
#     cv2.imshow("Frame", pred_image)
#     cv2.waitKey(0)

In [ ]:
print(prediction_status)